In [1]:
import re
import pymongo
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile,join
from pymongo.errors import BulkWriteError

In [2]:
#List of stations ids that have a prediction dataset. Just copy the path where you have locally stored the folder.
onlyfiles=[f for f in listdir('/home/usuario/ADS- Project/predictions') if f.endswith("_pred.csv")]
ids=[int(re.sub('_pred.csv','',text)) for text in onlyfiles]
ids.sort()

In [122]:
def station_info(station_id):    
    #Reading the useful columns of the historical dataset. Also reading 
    #chicago_stations for the information about the stations
    colnames1 = ['id', 'station_name', 'address', 'latitude', 'longitude']
    colnames2 = ['ts','total_docks']

    stats = pd.read_csv('chicago_stations.csv', names=colnames1,skiprows=[0])
    hist = pd.read_csv('data_output/id{}.csv'.format(station_id),names=colnames2,usecols=[1,3],skiprows=np.arange(0,145))
    
    stats_id=stats[stats.id==station_id] #info for the given station id
    docks=int(hist.total_docks.iloc[-1]) #number of total docks at the time the last historical measure was taken
    
    
    #Creating the dicctionary with all the information about the given station
    doc={'station_id': station_id,'station_name':stats_id['station_name'].values[0],'address':stats_id['address'].values[0],
         'latitude':stats_id['latitude'].values[0],'longitude':stats_id['longitude'].values[0],
         'total_docks':docks}
    
    #Connecting with mlab. Just check the folder with the credentials.
    try:
        with open("credentials/credentials.txt", 'r', encoding='utf-8') as f:
            [name,pasword,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,pasword, url,dbname)) 
    except pymongo.errors.ConnectionFailure as e:
        print ("Could not connect to MongoDB: %s" % e) 
    conn
    
    #Storing the dicctionary in mlab
    db = conn['easyrider2']
    collection = db['stations']
    collection.insert_one(doc)
    
    return doc

In [ ]:
#And just run the function defined above with all the stations for which we have predictions
for i in ids:
    station_info(i)

In [130]:
colnames2 = ['ts','available_bikes']
#Creating a dataset with all the historical and prediction data, adding station_id
i=0
for station_id in ids:    
    hist = pd.read_csv('data_output/id{}.csv'.format(station_id),names=colnames2,usecols=[1,6],skiprows=np.arange(0,145))
    pred = pd.read_csv('predictions/{}_pred.csv'.format(station_id), names=colnames2,usecols=[0,1],skiprows=[0])

    #Adding a column for station_id into the datasets
    hist['station_id']=station_id
    pred['station_id']=station_id
    #Sorting the columns to have the same order for both datasets
    cols = hist.columns.tolist()
    cols = [cols[0],cols[-1],cols[1]]
    hist = hist[cols]
    cols2= pred.columns.tolist()
    cols2= [cols2[0],cols2[-1],cols2[1]]
    pred = pred[cols2]
    #Merging historical and predicted in one dataframe
    frames = [hist,pred]
    result = pd.concat(frames)

    if i==0:
        dataset=result.copy()
    else:
        dataset=pd.concat([dataset,result])
    i+=1

ds=dataset.sort_values(by=['ts','station_id'])

In [131]:
#Creating a dictionary from the dataset
dic_ds=ds.to_dict('r')

In [125]:
try:
    with open("credentials/credentials.txt", 'r', encoding='utf-8') as f:
        [name,pasword,url,dbname]=f.read().splitlines()
    conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,pasword, url,dbname)) 
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
conn

#Connecting with mlab
db = conn['easyrider2']
collection = db['states']

In [ ]:
try:
    collection.insert_many(dic_ds)
except BulkWriteError as bwe:
    print(bwe.details)
    raise

In [128]:
collection.create_index([("ts", pymongo.ASCENDING)])

'ts_1'